In [1]:
from flask import Flask, render_template, request,jsonify
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
from pymongo import MongoClient

C:\Users\hites\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def index():
    searchString = "iphone7"
    try:
        client=MongoClient('mongodb://localhost:27017/')
        dbConn = pymongo.MongoClient("mongodb://localhost:27017/")  
        print(client.list_database_names())
        db = client["scraper"] 
        print(db.list_collection_names())
        London=db["London"]
        #reviews = db[searchString].find({})
        print( "----1")
        print(London.count_documents({}),"----2")
 #       if reviews.count_documents({}) > 0:
 #           return render_template('results.html',reviews=reviews) 
#      else:
        flipkart_url = "https://www.flipkart.com/search?q=" + searchString 
        print( "----3",flipkart_url)
        uClient = uReq(flipkart_url) 
        #print(uClient)
        flipkartPage = uClient.read() 
        #print(flipkartPage)
        uClient.close() 
        print( "----4")
        flipkart_html = bs(flipkartPage, "html.parser") 
        bigboxes = flipkart_html.findAll("div", {"class": "_1AtVbE col-12-12"}) 
        del bigboxes[0:3]
        print("---5")
        box = bigboxes[0] 
        print("---6")
        productLink = "https://www.flipkart.com" + box.div.div.div.a['href'] 
        print(productLink)
        prodRes = requests.get(productLink) 
        print("---7")
        prod_html = bs(prodRes.text, "html.parser") 
        commentboxes = prod_html.find_all('div', {'class': "_16PBlm"})
        print("-----8")

        table = db["London"] 
        filename = searchString+".csv" 
        print("----9")
        fw = open(filename, "w") 
        headers = "Product, Customer Name, Rating, Heading, Comment \n" 
        fw.write(headers) 
        print("----10")
        review = []
        for commentbox in commentboxes:
            try:
                name = commentbox.div.div.find_all('p', {'class': '_2sc7ZR _2V5EHH'})[0].text
                print("--for",name)
            except:
                name = 'No Name'
            try:
                rating = commentbox.div.div.div.div.text
                print("--rating",rating)
            except:
                rating = 'No Rating'
            try:
                commentHead = commentbox.div.div.div.p.text
                print("--comment",commentHead)
            except:
                commentHead = 'No Comment Heading'
            try:
                comtag = commentbox.div.div.find_all('div', {'class': ''})
                custComment = comtag[0].div.text
                print("----comtag",comtag)
            except:
                custComment = 'No Customer Comment'
            
            mydict = {"Product": searchString, "Name": name, "Rating": rating, "CommentHead": commentHead,"Comment": custComment} 
            print(mydict)
            x = table.insert_one(mydict) 
            review.append(mydict) 
 #       return render_template('results.html', reviews=reviews) 
    except:
        return 'something is wrong1'
    print(London.count_documents({}))

In [3]:
index()

['Batch-1', 'admin', 'config', 'demo', 'local', 'project']
['Paris', 'Scrapping project', 'London']
----1
3450 ----2
----3 https://www.flipkart.com/search?q=iphone7
----4
---5
---6
https://www.flipkart.com/apple-iphone-7-gold-32-gb/p/itmen6daf99nhhjz?pid=MOBEMK62HZHC6TFU&lid=LSTMOBEMK62HZHC6TFUZMLPAM&marketplace=FLIPKART&q=iphone7&store=tyy%2F4io&srno=s_1_2&otracker=search&fm=organic&iid=769465ae-b5b1-4e82-b60e-5ab0daabd90d.MOBEMK62HZHC6TFU.SEARCH&ppt=None&ppn=None&ssid=eo3mfjw0u80000001638716678637&qH=c848b7081ba96a14
---7
-----8
----9
----10
--for Hemanta Sa
--rating 5
--comment Worth every penny
----comtag [<div><div class="">Thank u Flipkart for your fast delivery. It is awesome and very very nice.I like this very much.</div><span class="_1H-bmy"><span>READ MORE</span></span></div>, <div class="">Thank u Flipkart for your fast delivery. It is awesome and very very nice.I like this very much.</div>, <div></div>]
{'Product': 'iphone7', 'Name': 'Hemanta Sa', 'Rating': '5', 'CommentHea